# Setup

### Model decisions

In [22]:
# Weights and Biases
!pip install -q wandb
# Tensorflow
!pip install -q tensorflow

In [23]:
previous_loading = False
google_colab = False
moneyness = False
lags = 80
hyperparameter_search = False
training_size = 991232
random_seed = 0
finance_computers = False
fc_path = 'M:/Master/'

In [24]:
if google_colab == True:
  from google.colab import drive
  drive.mount('/content/drive')

### Imports

In [25]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow import square, reduce_mean
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.math import multiply
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError
from math import log
import matplotlib.pyplot as plt
#import seaborn as sns
import matplotlib.ticker as ticker
from sklearn.metrics import mean_squared_error

In [26]:
# If running in colab, insert your wandb key here

#import config
#Erlend
wandb.login(key='3cae81eb56be3190be5bb48c571e69933071df69')
# Hjalmar
#wandb.login(key="b47bcf387a0571c5520c58a13be35cda8ada0a99")


wandb: Currently logged in as: erlendrygg (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/hjalmarjacobvinje/.netrc


True

# Load, split and normalize data

### Load data

In [27]:
if google_colab:
    import tensorflow as tf
    # Print info
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

    # Code to read csv file into Colaboratory:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "1Doyuyo_VDOmJf0CLo5kl9XzMTfhGtxiR"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('2010-2023_NSS_filtered_vF.csv')  
    df_read = pd.read_csv('2010-2023_NSS_filtered_vF.csv')
elif finance_computers:
    file = fc_path + "/data/processed_data/2010-2023_NSS_filtered_vF.csv"
    df_read = pd.read_csv(file)
else:
    file = "../data/processed_data/2010-2023_NSS_filtered_vF.csv"
    df_read = pd.read_csv(file)

display(df_read)

,Unnamed: 0,Quote_date,Price,Underlying_last,Strike,TTM,R
0,0,2010-01-04,207.490,1132.99,925.0,0.008219,0.00050
1,1,2010-01-04,182.500,1132.99,950.0,0.008219,0.00050
2,2,2010-01-04,157.500,1132.99,975.0,0.008219,0.00050
3,3,2010-01-04,132.600,1132.99,1000.0,0.008219,0.00050
4,4,2010-01-04,107.705,1132.99,1025.0,0.008219,0.00050
...,...,...,...,...,...,...,...
12057638,13739049,2023-03-31,217.750,4109.88,4700.0,1.726027,0.04198
12057639,13739050,2023-03-31,180.000,4109.88,4800.0,1.726027,0.04198
12057640,13739051,2023-03-31,146.550,4109.88,4900.0,1.726027,0.04198
12057641,13739052,2023-03-31,118.200,4109.88,5000.0,1.726027,0.04198


### Create lags

In [28]:
df = df_read
del df_read

if hyperparameter_search:
    df = df[df['Quote_date'] <= '2015-02-01']

# Group the data by Quote Date and calculate the mean for Underlying Price
df_agg = df.groupby('Quote_date').mean().reset_index()

# Values to returns
df_agg["Underlying_return"] = df_agg["Underlying_last"].pct_change()

# Add the Underlying Price Lag column
for i in range(1, lags + 1):
    df_agg['Underlying_' + str(i)] = df_agg['Underlying_return'].shift(i)

df = pd.merge(df, df_agg[['Quote_date', 'Underlying_return'] + ['Underlying_' + str(i) for i in range(1, lags + 1)]], on='Quote_date', how='left')

# Filter df between 2014-01-01 and 2022-12-31
df = df[(df["Quote_date"] >= "2011-09-01") & (df["Quote_date"] <= "2022-12-31")]

print(df)

          Unnamed: 0  Quote_date    Price  Underlying_last  Strike       TTM  \
292779        381812  2011-09-01  304.355          1203.96   900.0  0.002740   
292780        381813  2011-09-01  279.345          1203.96   925.0  0.002740   
292781        381814  2011-09-01  254.355          1203.96   950.0  0.002740   
292782        381815  2011-09-01  229.350          1203.96   975.0  0.002740   
292783        381816  2011-09-01  204.350          1203.96  1000.0  0.002740   
...              ...         ...      ...              ...     ...       ...   
11654831    13280615  2022-12-30  362.600          3839.81  4300.0  1.975342   
11654832    13280616  2022-12-30  319.150          3839.81  4400.0  1.975342   
11654833    13280617  2022-12-30  279.000          3839.81  4500.0  1.975342   
11654834    13280618  2022-12-30  241.950          3839.81  4600.0  1.975342   
11654835    13280619  2022-12-30  208.800          3839.81  4700.0  1.975342   

                 R  Underlying_return  

### Format input data

In [29]:
# Format settings
max_timesteps = lags
bs_vars = ['Moneyness', 'TTM', 'R'] if moneyness else ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = [f'Underlying_{i}' for i in range (max_timesteps - 1, 0, -1)] + ['Underlying_return']

def create_rw_dataset(window_number = 0, df = None):
    '''Creates dataset for a single rolling window period offsett by the window number'''

    # Create train, validation and test set split points
    test_months = 3
    train_start = datetime(2011,9,1) + relativedelta(months=window_number * test_months)
    val_start = train_start + relativedelta(months=3*12)
    test_start = val_start + relativedelta(months = 1 + test_months) if hyperparameter_search else val_start + relativedelta(months = 1)
    test_end = test_start + relativedelta(months=test_months)
    train_start = str(train_start.date())
    val_start = str(val_start.date())
    test_start = str(test_start.date())
    test_end = str(test_end.date())

        
    # Split train and validation data
    df_train = df[(df['Quote_date'] >= train_start) & (df['Quote_date'] < val_start)]
    df_val = df[(df['Quote_date'] >= val_start) & (df['Quote_date'] < test_start)]
    df_test = df[(df['Quote_date'] >= test_start) & (df['Quote_date'] < test_end)]

    del df

    # Extract target values
    train_y = (df_train['Price'] / df_train['Strike']).to_numpy() if moneyness else df_train['Price'].to_numpy()
    val_y = (df_val['Price'] / df_val['Strike']).to_numpy() if moneyness else df_val['Price'].to_numpy()
    test_y = (df_test['Price'] / df_test['Strike']).to_numpy() if moneyness else df_test['Price'].to_numpy()

    # If usining moneyness, extract strike
    if moneyness:
        train_strike = df_train['Strike'].to_numpy()
        val_strike = df_val['Strike'].to_numpy()
        test_strike = df_test['Strike'].to_numpy()


    # Print earliest and latest date in every dataframe used
    print("--------------Dataframe dates--------------")
    print(f"Train: {df_train['Quote_date'].min()} - {df_train['Quote_date'].max()}")
    print(f"Val: {df_val['Quote_date'].min()} - {df_val['Quote_date'].max()}")
    print(f"Test: {df_test['Quote_date'].min()} - {df_test['Quote_date'].max()}")
    print("-------------------------------------------")

    # Convert dataframes to numpy arrays
    train_x = [df_train[underlying_lags].to_numpy(), df_train[bs_vars].to_numpy()]
    val_x = [df_val[underlying_lags].to_numpy(), df_val[bs_vars].to_numpy()]
    test_x = [df_test[underlying_lags].to_numpy(), df_test[bs_vars].to_numpy()]

    del df_train
    del df_val

    # Scale features based on training set
    underlying_scaler = MinMaxScaler()
    train_x[0] = underlying_scaler.fit_transform(train_x[0].flatten().reshape(-1, 1)).reshape(train_x[0].shape)
    val_x[0] = underlying_scaler.transform(val_x[0].flatten().reshape(-1,1)).reshape(val_x[0].shape)
    test_x[0] = underlying_scaler.transform(test_x[0].flatten().reshape(-1,1)).reshape(test_x[0].shape)

    bs_scaler = MinMaxScaler()
    train_x[1] = bs_scaler.fit_transform(train_x[1])
    val_x[1] = bs_scaler.transform(val_x[1])
    test_x[1] = bs_scaler.transform(test_x[1])


    # Shuffle training set
    np.random.seed(random_seed)
    shuffle = np.random.permutation(len(train_x[0]))
    train_x = [train_x[0][shuffle], train_x[1][shuffle]]
    train_y = train_y[shuffle]

    # Extract training set
    train_x = [train_x[0][:training_size], train_x[1][:training_size]]
    train_y = train_y[:training_size]

    if moneyness:
        train_strike = train_strike[shuffle][:training_size]

    # Reshape data to fit LSTM
    train_x = [train_x[0].reshape(len(train_x[0]), max_timesteps, 1), train_x[1]]
    val_x = [val_x[0].reshape(len(val_x[0]), max_timesteps, 1), val_x[1]]
    test_x = [test_x[0].reshape(len(test_x[0]), max_timesteps, 1), test_x[1]]

    print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
    print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')
    print(f'Test shape: {test_x[0].shape}, {test_x[1].shape}')

    if moneyness:
        return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike,
    return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test

# Create the dataset for the first rolling window period
if moneyness:
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike = create_rw_dataset(df=df)
else:
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(df=df)
    if hyperparameter_search:
        del df

--------------Dataframe dates--------------
Train: 2011-09-01 - 2014-08-29
Val: 2014-09-02 - 2014-09-30
Test: 2014-10-01 - 2014-12-31
-------------------------------------------
Train shape: (991232, 80, 1), (991232, 4)
Val shape: (54323, 80, 1), (54323, 4)
Test shape: (177109, 80, 1), (177109, 4)


In [30]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

def compute_saliency_map(model, input_data, percentage=0.02):
    n_samples = len(input_data[0])
    sample_size = int(n_samples * percentage)
    
    # Randomly select a subset of samples
    idx = np.random.choice(n_samples, sample_size, replace=False)
    input_data_underlying_sample = tf.convert_to_tensor(input_data[0][idx], dtype=tf.float32)
    input_data_bs_sample = tf.convert_to_tensor(input_data[1][idx], dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(input_data_underlying_sample)
        tape.watch(input_data_bs_sample)
        output = model([input_data_underlying_sample, input_data_bs_sample])

    grads = tape.gradient(output, [input_data_underlying_sample, input_data_bs_sample])
    grads_underlying = grads[0]
    grads_bs = grads[1]

    saliency_map_underlying = np.mean(np.abs(grads_underlying.numpy()), axis=0)
    saliency_map_bs = np.mean(np.abs(grads_bs.numpy()), axis=0)

    return saliency_map_underlying, saliency_map_bs



input_data = train_x  # or use val_x


In [31]:
model = load_model(f'model.h5')


In [32]:
saliency_map_underlying, saliency_map_bs = compute_saliency_map(model, input_data)

In [33]:
print(f'saliency_map_underlying shape: {saliency_map_underlying.shape}')
print(f'saliency_map_bs shape: {saliency_map_bs.shape}')
saliency_map_bs = np.reshape(saliency_map_bs, (-1, 1))
print(f'saliency_map_bs shape: {saliency_map_bs.shape}')
print(saliency_map_bs)
print(saliency_map_underlying)

# Sum of saliency map for underlying
saliency_map_underlying_sum = np.sum(saliency_map_underlying)
print(f'The sum of the saliency map for the underlying lags is: {saliency_map_underlying_sum}')
# Sum of saliency map for bs
saliency_map_bs_sum = np.sum(saliency_map_bs)
print(f'The sum of the saliency map for the bs is: {saliency_map_bs_sum}')
print("The proportion of the saliency map for the underlying lags is: ", saliency_map_underlying_sum / (saliency_map_underlying_sum + saliency_map_bs_sum))

saliency_map_underlying shape: (80, 1)
saliency_map_bs shape: (4,)
saliency_map_bs shape: (4, 1)
[[1822.6469 ]
 [3314.0334 ]
 [ 360.9834 ]
 [  74.55522]]
[[ 1.4579371]
 [ 4.632706 ]
 [ 8.211611 ]
 [10.954626 ]
 [13.114001 ]
 [14.92432  ]
 [16.41222  ]
 [17.71966  ]
 [18.881113 ]
 [19.906038 ]
 [20.848652 ]
 [21.67782  ]
 [22.499826 ]
 [23.22376  ]
 [23.929852 ]
 [24.614902 ]
 [25.258356 ]
 [25.846985 ]
 [26.473284 ]
 [27.029541 ]
 [27.569786 ]
 [28.071095 ]
 [28.575287 ]
 [29.042067 ]
 [29.482733 ]
 [29.938616 ]
 [30.325136 ]
 [30.723433 ]
 [31.083633 ]
 [31.425888 ]
 [31.72545  ]
 [31.976746 ]
 [32.22938  ]
 [32.421623 ]
 [32.65192  ]
 [32.810394 ]
 [32.938778 ]
 [33.00454  ]
 [33.06929  ]
 [33.0878   ]
 [33.07341  ]
 [33.026188 ]
 [32.93582  ]
 [32.813416 ]
 [32.645084 ]
 [32.39636  ]
 [32.136818 ]
 [31.83548  ]
 [31.447115 ]
 [31.029903 ]
 [30.567327 ]
 [30.056103 ]
 [29.466026 ]
 [28.863064 ]
 [28.197355 ]
 [27.470951 ]
 [26.754515 ]
 [25.97157  ]
 [25.220943 ]
 [24.452072 ]
 [23.6

In [34]:
# Saliency map for underlying history
bs_variables = ['Underlying', 'Strike Price', 'Time to Maturity', 'Rent']
plt.imshow(saliency_map_bs, cmap='viridis', aspect='auto')
# Create axis ticks
ax = plt.gca()
plt.colorbar()
plt.xlabel('Timesteps')
plt.ylabel('Features')
plt.title('Saliency Map - Underlying History')
plt.show()

# Saliency map for underlying history
lag_steps = [x for x in range(-max_timesteps, 0)]
plt.imshow(saliency_map_underlying, cmap='viridis', aspect='auto')
plt.colorbar()
plt.yticks(range(len(lag_steps)), lag_steps)
#ax = plt.gca()
#ax.yaxis.set_major_locator(ticker.MultipleLocator(base=5))
plt.xlabel('Timesteps')
plt.ylabel('Features')
plt.show()

SystemError: PY_SSIZE_T_CLEAN macro must be defined for '#' formats

<Figure size 640x480 with 2 Axes>

SystemError: PY_SSIZE_T_CLEAN macro must be defined for '#' formats

<Figure size 640x480 with 2 Axes>